# 📊 Actividad 1 — Big Data  
**Autores:** Indira Hamdam – Cristian Vicioso  
**Programa:** Big Data 
**Plataforma:** Databricks Free Edition  

## 🎯 Objetivo
Aplicar los conceptos de la Unidad 1 para formular una problemática real que requiera una base de datos analítica, diseñar el modelo entidad–relación, crear la base de datos, insertar información y documentar el proceso dentro de un Jupyter Notebook en Databricks.

## Definición de la problemática

**Título:**  
📈 *Análisis de la calidad del aire y sus efectos en la salud respiratoria en las ciudades de Medellín y Santa Marta, Colombia.*

**Descripción:**  
El proyecto busca analizar cómo los niveles de contaminación del aire (PM2.5, PM10, NO₂) impactan los casos de enfermedades respiratorias en Medellín y Santa Marta. El análisis permitirá generar visualizaciones y comparaciones temporales para la toma de decisiones ambientales y sanitarias.

**Dataset seleccionado:**  
[Air Quality Dataset - Kaggle](https://www.kaggle.com/datasets/fedesoriano/air-quality-data-set)

**Variables relevantes:**

| Variable | Descripción | Tipo de dato | Utilidad |
|-----------|--------------|---------------|----------|
| city | Ciudad | string | Filtra por ciudad |
| datetime | Fecha | date | Análisis temporal |
| PM2.5 | Contaminante fino | float | Afecta salud respiratoria |
| PM10 | Contaminante grueso | float | Control ambiental |
| NO2 | Dióxido de nitrógeno | float | Contaminación vehicular |
| temperature | Temperatura | float | Contexto climático |
| humidity | Humedad relativa | float | Factores ambientales |
| respiratory_cases | Casos respiratorios | int | Indicador de salud |

## Diseño del modelo Entidad–Relación (ERD)

El modelo contiene dos entidades principales:

**CIUDAD**  
- id_ciudad *(PK)*  
- nombre  
- departamento  

**MEDICION_AMBIENTAL**  
- id_medicion *(PK)*  
- id_ciudad *(FK)*  
- fecha  
- pm25  
- pm10  
- no2  
- temperatura  
- humedad  
- casos_respiratorios  

### 🔗 Relación
Una ciudad puede registrar muchas mediciones relación **1:N**

In [0]:
er_diagram = """
CIUDAD {
    int id_ciudad PK
    string nombre
    string departamento
}
MEDICION_AMBIENTAL {
    int id_medicion PK
    int id_ciudad FK
    date fecha
    float pm25
    float pm10
    float no2
    float temperatura
    float humedad
    int casos_respiratorios
}
CIUDAD ||--o{ MEDICION_AMBIENTAL : "registra"
"""

## 📊 Diagrama Entidad–Relación (ERD)

![Modelo ERD](/Workspace/Users/indira.hamdam@est.iudigital.edu.co/modelo_ERD.png)

**Descripción:**  
El modelo representa dos entidades: `Ciudad` y `Medicion_Ambiental`.  
Cada ciudad puede tener múltiples mediciones de calidad del aire (relación 1:N).  
Esto permite analizar la contaminación ambiental y su relación con los casos de enfermedades respiratorias por ciudad y fecha.

## Creación de la base de datos e inserción de información

En esta sección se crean las tablas **Ciudad** y **Medicion_Ambiental** utilizando **Spark SQL** en la plataforma **Databricks Community Edition (Free)**.

Las tablas se definen como *vistas temporales* (`TEMP VIEW`), lo que permite tratarlas como una base de datos relacional dentro del entorno de Big Data.

A continuación se muestran las sentencias `CREATE TABLE` e `INSERT INTO` equivalentes, ejecutadas con `spark.sql()`, seguidas de un ejemplo de consulta para validar la carga de datos.

In [0]:
# Crear las tablas en Spark SQL
spark.sql("""
CREATE OR REPLACE TEMP VIEW Ciudad AS
SELECT 1 AS id_ciudad, 'Medellín' AS nombre, 'Antioquia' AS departamento
UNION ALL
SELECT 2, 'Santa Marta', 'Magdalena'
""")

spark.sql("""
CREATE OR REPLACE TEMP VIEW Medicion_Ambiental AS
SELECT 1 AS id_medicion, 1 AS id_ciudad, DATE('2023-05-01') AS fecha, 35.2 AS pm25, 55.1 AS pm10, 27.4 AS no2, 22.5 AS temperatura, 68 AS humedad, 120 AS casos_respiratorios
UNION ALL
SELECT 2, 2, DATE('2023-05-01'), 22.3, 40.2, 15.7, 30.1, 75, 65
UNION ALL
SELECT 3, 1, DATE('2023-05-02'), 40.5, 59.3, 30.2, 23.1, 70, 132
""")

print("Tablas creadas correctamente ✅")


Tablas creadas correctamente ✅


In [0]:
spark.sql("SELECT * FROM Medicion_Ambiental LIMIT 5").show()

+-----------+---------+----------+----+----+----+-----------+-------+-------------------+
|id_medicion|id_ciudad|     fecha|pm25|pm10| no2|temperatura|humedad|casos_respiratorios|
+-----------+---------+----------+----+----+----+-----------+-------+-------------------+
|          1|        1|2023-05-01|35.2|55.1|27.4|       22.5|     68|                120|
|          2|        2|2023-05-01|22.3|40.2|15.7|       30.1|     75|                 65|
|          3|        1|2023-05-02|40.5|59.3|30.2|       23.1|     70|                132|
+-----------+---------+----------+----+----+----+-----------+-------+-------------------+



## Evidencias de la base de datos con consultas SQL

En esta sección se muestran las consultas realizadas para comprobar la correcta creación y comprensión de la base de datos.  
Se utiliza el motor **Spark SQL** de **Databricks Free Edition**.

In [0]:
# Conteo de registros
spark.sql("SELECT COUNT(*) AS total_registros FROM Medicion_Ambiental").show()

+---------------+
|total_registros|
+---------------+
|              3|
+---------------+



In [0]:
# Nombres y tipos de columnas
spark.sql("DESCRIBE TABLE Medicion_Ambiental").show(truncate=False)

+-------------------+------------+-------+
|col_name           |data_type   |comment|
+-------------------+------------+-------+
|id_medicion        |int         |NULL   |
|id_ciudad          |int         |NULL   |
|fecha              |date        |NULL   |
|pm25               |decimal(3,1)|NULL   |
|pm10               |decimal(3,1)|NULL   |
|no2                |decimal(3,1)|NULL   |
|temperatura        |decimal(3,1)|NULL   |
|humedad            |int         |NULL   |
|casos_respiratorios|int         |NULL   |
+-------------------+------------+-------+



In [0]:
# Consulta filtrada
spark.sql("""
SELECT *
FROM Medicion_Ambiental
WHERE id_ciudad = 1
""").show()

+-----------+---------+----------+----+----+----+-----------+-------+-------------------+
|id_medicion|id_ciudad|     fecha|pm25|pm10| no2|temperatura|humedad|casos_respiratorios|
+-----------+---------+----------+----+----+----+-----------+-------+-------------------+
|          1|        1|2023-05-01|35.2|55.1|27.4|       22.5|     68|                120|
|          3|        1|2023-05-02|40.5|59.3|30.2|       23.1|     70|                132|
+-----------+---------+----------+----+----+----+-----------+-------+-------------------+



**Interpretación:**  
Esta consulta filtra las mediciones correspondientes a la ciudad de **Medellín (id_ciudad = 1)**.  
Se observan valores más altos de PM2.5 y PM10 en comparación con Santa Marta, lo que sugiere una mayor concentración de contaminantes en entornos urbanos más densos.

**Interpretación:**  
La tabla `Medicion_Ambiental` contiene el número total de registros insertados.  
Este valor confirma que la carga inicial de datos se realizó correctamente.  
En este caso, se cuentan 3 registros de mediciones ambientales (una pequeña muestra de prueba).

In [0]:
spark.sql("""
SELECT c.nombre AS ciudad, COUNT(m.id_medicion) AS total_mediciones
FROM Medicion_Ambiental m
JOIN Ciudad c ON c.id_ciudad = m.id_ciudad
GROUP BY c.nombre
""").show()

+-----------+----------------+
|     ciudad|total_mediciones|
+-----------+----------------+
|   Medellín|               2|
|Santa Marta|               1|
+-----------+----------------+



**Interpretación:**  
Esta consulta agrupa las mediciones por ciudad, mostrando cuántos registros tiene cada una.  
Permite confirmar que la relación entre las tablas `Ciudad` y `Medicion_Ambiental` funciona correctamente.

## Conclusiones

- El modelo entidad–relación permitió vincular mediciones ambientales y casos de salud.
- Los datos sugieren una posible correlación entre PM2.5/PM10 y enfermedades respiratorias.
- Databricks facilita el análisis distribuido y consultas SQL escalables.
- Este notebook puede ampliarse con datos reales de fuentes como el IDEAM o el Ministerio de Salud.